#### CREATE STREAMING TABLE FOR `CUSTOMERS.JSON` AND LOAD INTO `BRONZE_CUSTOMERS`

In [0]:
CREATE OR REFRESH STREAMING TABLE BRONZE_CUSTOMERS
COMMENT 'Raw customer data ingested from cloud storage volume'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT
*,
_metadata.file_path AS file_path,
_metadata.file_name AS file_name,
CURRENT_TIMESTAMP AS load_timestamp
FROM
cloud_files('/Volumes/circuitbox_dev/landing/operational_data/customers/', 
'json',
map('cloudFiles.inferColumnTypes', 'true')
)

#### PERFORM DATA QUALITY CHECKS
- FAIL IS `CUSTOMER ID` IS NULL
- DROP RECORDS WHEN `CUSTOMER NAME` IS NULL
- WARN IF 'TELEPHONE NUMBER' IS LESS THAN 10 CHARACTERS
- WARN IF `EMAIL` IS NULL
- WARN IF `DOB` IS BEFORE 1920

In [0]:
CREATE OR REFRESH STREAMING TABLE SILVER_CUSTOMERS
(
CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
CONSTRAINT valid_customer_name EXPECT (customer_name IS NOT NULL) ON VIOLATION DROP ROW,
CONSTRAINT valid_customer_telephone EXPECT (LENGTH(telephone) >= 10),
CONSTRAINT valid_customer_email EXPECT (email IS NOT NULL),
CONSTRAINT valid_date_of_birth EXPECT (date_of_birth >= '1920-01-01')
)
COMMENT 'Cleaned customer data ingested from cloud storage volume'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT
customer_id,
customer_name,
CAST(date_of_birth AS DATE) AS date_of_birth,
telephone,
email,
CAST(created_date AS DATE) AS created_date
FROM
STREAM(LIVE.BRONZE_CUSTOMERS);

#### PERFORM SLOWLY CHANGING DIMENSION (SCD I) FOR `CUSTOMERS`

In [0]:
CREATE OR REFRESH STREAMING TABLE SCD_1_CUSTOMERS
COMMENT 'SCD TYPE 1'
TBLPROPERTIES ('quality' = 'gold')

In [0]:
APPLY CHANGES INTO LIVE.SCD_1_customers
FROM STREAM(LIVE.SILVER_CUSTOMERS)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1;

#### PERFORM SLOWLY CHANGING DIMENSION (SCD 2) FOR `CUSTOMERS`

In [0]:
CREATE OR REFRESH STREAMING TABLE SCD_2_CUSTOMERS
COMMENT 'SCD TYPE 2'
TBLPROPERTIES ('quality' = 'gold')

In [0]:
APPLY CHANGES INTO LIVE.SCD_2_CUSTOMERS
FROM STREAM(LIVE.SILVER_CUSTOMERS)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 2;